In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os
import warnings
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')    # 警告メッセージを無視

In [3]:
# ========== 初期設定 (初始化设置) ==========

# pandasですべての列を表示するように設定（省略なし）
pd.set_option('display.max_columns', None) 
# 乱数シードを42に固定（再現性のため）
np.random.seed(42) 

# 保存パスの設定読み込み
DATA_PATH = '测试数据.xlsx'
OUTPUT_PATH = '测试数据预处理结果.xlsx'  

In [4]:
# ========== データの読み込みと検証 (数据加载与校验) ==========

# データ読み込み
df = pd.read_excel(DATA_PATH)
# データ読み込み成功、元の形状を表示
print(f"数据加载成功，原始形状：{df.shape}") #データの読み込みに成功、元の形状

数据加载成功，原始形状：(9, 108)


In [5]:
# 不要な列の定義と削除
# データクレンジングのプロセスでは、分析に不要な識別子や個人情報を削除します


# CLLI_OID 一意識別子、特徴の汎化能力なし
# CL_NO 請求番号、純粋な管理番号 
# LINE_NO 請求明細行番号、分析価値なし
# FX_RATE 為替レート（国内業務は1に固定）、実質的な意味なし  
# SCMA_OID_BEN_TYPE BEN_TYPEと重複、冗長フィールド 
# BARCODE 個別案件コード、一意識別子
# PRI_CORR_BRKR_NAME 固定値“BSI”、分析価値なし
# CRT_USER 作成者、ユーザー行動とは無関係
# UPD_USER 更新者、ユーザー行動とは無関係
# ID_CARD_NO 身分証番号、プライバシーコンプライアンスリスクあり
# PHONE_NO 電話番号、プライバシーコンプライアンスリスクあり
# PAYEE_LAST_NAME 受取人姓、プライバシーコンプライアンスリスクあり 
# PAYEE_FIRST_NAME 受取人名、プライバシーコンプライアンスリスクあり
# LINE_REMARK 非構造化テキスト、NLP処理が必要で利益は不確定                    
# CSR_REMARK 非構造化テキスト、NLP処理が必要で利益は不確定                    
# PLAN_REMARK 非構造化テキスト、NLP処理が必要で利益は不確定

cols_to_drop = ['CLLI_OID', 'CL_NO', 'LINE_NO', 'BARCODE', 'FX_RATE','PRI_CORR_BRKR_NAME','SCMA_OID_BEN_TYPE', 'CRT_USER', 'UPD_USER',
                'ID_CARD_NO', 'PHONE_NO', 'PAYEE_LAST_NAME', 'PAYEE_FIRST_NAME','CL_PAY_ACCT_NO','POCY_REF_NO','MBR_REF_NO','CLSH_HOSP_CODE',
                'LINE_REMARK', 'CSR_REMARK', 'PLAN_REMARK','MAN_REJ_CODE_DESC_','CL_LINE_FORMULA','CL_CLAIM_FORMULA','CL_INVOICE_FORMULA',
               'KIND_CODE','MAN_REJ_CODE_DESC_1','MAN_REJ_CODE_DESC_2','BEN_HEAD_TYPE','MBR_REF_NO_B','ORG_INSUR_INVOICE_IND','FILE_ID'
               'MEPL_MBR_REF_NO','MEPL_MBR_REF_NO_B','MBR_LAST_NAME  ','BANK_NAME','CL_PAY_ACCT_NAME','FILE_ID ','MAN_REJ_AMT_2',
               'FILE_CLOSE_DATE','TOTAL_RECEIPT_AMT','MEPL_MBR_REF_NO','MAN_REJ_AMT_1','PROV_DEPT','MBR_LAST_NAME','FILE_ID','WORKPLACE_NAME',
                'POCY_PLAN_DESC','INCUR_DATE_FROM','INCUR_DATE_TO','PAY_DATE','CRT_DATE','UPD_DATE','DIAG_DESC','SCMA_OID_CL_LINE_STATUS',
               'RCV_DATE','MBR_FIRST_NAME','SCMA_OID_PROD_TYPE','SCMA_OID_CL_STATUS','SCMA_OID_CL_TYPE','SCMA_OID_COUNTRY_TREATMENT','MEMBER_EVENT',
               'INSUR_INVOICE_IND','PROV_NAME','MBR_TYPE','BOX_BARCODE','PAY_AMT','STR_CRT_DATE','ORG_PRES_AMT','PROV_CODE','MBR_NO','STR_UPD_DATE','POHO_NO',
                'POPL_OID','INVOICE_ID','CL_LINE_NO','PLAN_OID','POCY_NO','POLICY_CNT','INVOICE_NO'
               ,'BEN_HEAD','RJ_CODE_LIST','RECHARGE_AMT']
# データフレームに存在する列のみを削除リストに保持
cols_to_drop = [col for col in cols_to_drop if col in df.columns]
df.drop(columns=cols_to_drop, inplace=True) #inplace=True表示直接在原DataFrame上修改
print(f"已删除字段：{cols_to_drop}") #削除されたフィールド


# 2. 目的変数のエンコーディング
# 詐欺フラグの変換: 'AC' → 0（非詐欺/正常請求）, 'RJ' → 1（詐欺/拒絶/疑義）, 'PD'/'PV' → 1
if 'CL_LINE_STATUS' in df.columns:
    df['fraud'] = df['CL_LINE_STATUS'].map({'AC': 0, 'RJ': 1,'PD':1,'PV':1})
    df.drop(columns=['CL_LINE_STATUS'], inplace=True)

# すべてが空値の列を削除
empty_cols = df.columns[df.isnull().all()].tolist()
if empty_cols:
    df.drop(empty_cols, axis=1, inplace=True)
    print(f"已删除全空列：{empty_cols}") #すべて空のため削除された列

已删除字段：['CLLI_OID', 'CL_NO', 'LINE_NO', 'BARCODE', 'FX_RATE', 'PRI_CORR_BRKR_NAME', 'SCMA_OID_BEN_TYPE', 'CRT_USER', 'UPD_USER', 'ID_CARD_NO', 'PHONE_NO', 'PAYEE_LAST_NAME', 'PAYEE_FIRST_NAME', 'CL_PAY_ACCT_NO', 'POCY_REF_NO', 'MBR_REF_NO', 'CLSH_HOSP_CODE', 'LINE_REMARK', 'CSR_REMARK', 'PLAN_REMARK', 'CL_LINE_FORMULA', 'CL_CLAIM_FORMULA', 'CL_INVOICE_FORMULA', 'KIND_CODE', 'MAN_REJ_CODE_DESC_1', 'MAN_REJ_CODE_DESC_2', 'BEN_HEAD_TYPE', 'MBR_REF_NO_B', 'ORG_INSUR_INVOICE_IND', 'MEPL_MBR_REF_NO_B', 'BANK_NAME', 'CL_PAY_ACCT_NAME', 'MAN_REJ_AMT_2', 'FILE_CLOSE_DATE', 'TOTAL_RECEIPT_AMT', 'MEPL_MBR_REF_NO', 'MAN_REJ_AMT_1', 'PROV_DEPT', 'MBR_LAST_NAME', 'FILE_ID', 'WORKPLACE_NAME', 'POCY_PLAN_DESC', 'INCUR_DATE_FROM', 'INCUR_DATE_TO', 'PAY_DATE', 'CRT_DATE', 'UPD_DATE', 'DIAG_DESC', 'SCMA_OID_CL_LINE_STATUS', 'RCV_DATE', 'MBR_FIRST_NAME', 'SCMA_OID_PROD_TYPE', 'SCMA_OID_CL_STATUS', 'SCMA_OID_CL_TYPE', 'SCMA_OID_COUNTRY_TREATMENT', 'MEMBER_EVENT', 'INSUR_INVOICE_IND', 'PROV_NAME', 'MBR_TYPE'

In [6]:
from sklearn.preprocessing import MinMaxScaler

# 数値型列の選択（目的変数 'fraud' を除く）
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
num_cols = [col for col in num_cols if col != 'fraud']

# MinMaxScalerの初期化（データを0から1の範囲に正規化）
scaler = MinMaxScaler()

# 選択された列に対してMin-Max正規化を実行
df[num_cols] = scaler.fit_transform(df[num_cols])

# 正規化後のデータ範囲を確認
print("Min-Max 标准化后的数据范围：") #Min-Max正規化後のデータ範囲
print(df[num_cols].describe().loc[['min', 'max']])


Min-Max 标准化后的数据范围：
     ORG_PRES_AMT_VALUE  APP_AMT  BEN_SPEND  PAY_AMT_USD  REJECTED_AMT  \
min                 0.0      0.0        0.0          0.0           0.0   
max                 1.0      1.0        1.0          1.0           1.0   

     SUB_AMT  CL_SOCIAL_PAY_AMT  CL_THIRD_PARTY_PAY_AMT  CL_OWNER_PAY_AMT  \
min      0.0                0.0                     0.0               0.0   
max      1.0                1.0                     0.0               1.0   

     CL_SELF_CAT_PAY_AMT  COPAY_PCT  CWF_AMT_DAY  DED_AMT  NO_OF_YR  \
min                  0.0        0.0          0.0      0.0       0.0   
max                  1.0        1.0          0.0      0.0       0.0   

     INVOICE_CNT  
min          0.0  
max          1.0  


In [7]:
# 特徴量エンコーディング (カテゴリカル変数の数値化)
from sklearn.preprocessing import LabelEncoder

# PROV_LEVEL（プロバイダーレベル）のエンコーディング
if 'PROV_LEVEL' in df.columns:
    # LabelEncoderの初期化
    label_encoder = LabelEncoder()
    
    # エンコーディングの実行
    df['PROV_LEVEL'] = label_encoder.fit_transform(df['PROV_LEVEL'])
    
    # エンコーディングのマッピング関係を出力
    print("PROV_LEVEL 编码映射关系：") #PROV_LEVEL エンコーディングマッピング関係
    for i, label in enumerate(label_encoder.classes_):
        print(f"{label} → {i}")



# BEN_TYPE（給付タイプ）のエンコーディング
if 'BEN_TYPE' in df.columns:
    # 一意な値の統計
    unique_ben_types = df['BEN_TYPE'].unique()
    print(f"BEN_TYPE 的唯一值有 {len(unique_ben_types)} 种，具体为：{unique_ben_types}")
    
    # LabelEncoderの初期化と適用
    label_encoder = LabelEncoder()
    
    
    df['BEN_TYPE'] = label_encoder.fit_transform(df['BEN_TYPE'])
    
    # マッピング関係の出力
    print("BEN_TYPE 编码映射关系：")
    for i, label in enumerate(label_encoder.classes_):
        print(f"{label} → {i}")



# DIAG_CODE（診断コード）のエンコーディング（最初の1文字に基づく）
if 'DIAG_CODE' in df.columns:
    # 分類基準として最初の1文字を抽出
    df['DIAG_CODE_PREFIX'] = df['DIAG_CODE'].str[:1]
    
    # プレフィックスの一意な値を確認
    unique_prefixes = df['DIAG_CODE_PREFIX'].unique()
    print(f"DIAG_CODE 的前1个字符有 {len(unique_prefixes)} 种，具体为：{unique_prefixes}")
    
    # LabelEncoderの初期化と適用
    label_encoder = LabelEncoder()
    df['DIAG_CODE_PREFIX'] = label_encoder.fit_transform(df['DIAG_CODE_PREFIX'])
    
    # マッピング関係の出力
    print("DIAG_CODE 前1个字符编码映射关系：")
    for i, label in enumerate(label_encoder.classes_):
        print(f"{label} → {i}")
    
    # マッピング関係の出力
    df.drop(columns=['DIAG_CODE'], inplace=True)


# 1. CODES列の存在確認
if 'CODES' in df.columns:
    # 2. 欠損値を埋めて文字列型に変換
    df['CODES'] = df['CODES'].fillna('').astype(str)
    
    # 3. カンマ区切りの項目数をカウントして新しい特徴量とする
    df['CODES_COUNT'] = df['CODES'].apply(lambda x: len(x.split(',')) if x else 0)
    
    # 4. 元のCODES列を削除
    df.drop(columns=['CODES'], inplace=True)
    
    print("CODES 列已编码为字符串数量，新增列：CODES_COUNT")
    #CODES列は項目数にエンコードされました、追加列：CODES_COUNT
else:
    print("数据中未找到 CODES 列，请检查数据！")
    #データ内にCODES列が見つかりません。データを確認してください！

PROV_LEVEL 编码映射关系：
三级 → 0
二级 → 1
BEN_TYPE 的唯一值有 1 种，具体为：['OP']
BEN_TYPE 编码映射关系：
OP → 0
DIAG_CODE 的前1个字符有 6 种，具体为：['K' 'G' 'L' 'E' 'M' 'R']
DIAG_CODE 前1个字符编码映射关系：
E → 0
G → 1
K → 2
L → 3
M → 4
R → 5
CODES 列已编码为字符串数量，新增列：CODES_COUNT


In [8]:
# ========== データ出力 (数据输出) ==========

# 出力ディレクトリが存在することを確認
os.makedirs(os.path.dirname(OUTPUT_PATH) or '.', exist_ok=True)

# 処理後のデータをExcelファイルとして保存
df.to_excel(OUTPUT_PATH, index=False)
print(f"\n数据处理完成，结果已保存到: {os.path.abspath(OUTPUT_PATH)}")
#データ処理完了、結果保存先
print(f"最终数据形状：{df.shape}")
#最終データ形状


数据处理完成，结果已保存到: C:\Users\27826\Desktop\项目\4.结果预测\测试数据预处理结果.xlsx
最终数据形状：(9, 20)


In [9]:
pip install pandas xgboost joblib openpyxl

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import xgboost as xgb
import joblib

In [11]:
# 前処理済みのデータを読み込む
df = pd.read_excel('测试数据预处理结果.xlsx')

# データの形状を確認
print(df.shape)  

(9, 20)


In [12]:
# ========== 特徴量の動的調整 (动态调整特征) ==========

# モデルの学習時に使用された特徴量リスト
expected_columns = ['PROV_LEVEL', 'INVOICE_CNT', 'CL_THIRD_PARTY_PAY_AMT', 'CWF_AMT_DAY', 'CODES_COUNT', 
                    'CL_OWNER_PAY_AMT', 'PAY_AMT_USD', 'APP_AMT', 'BEN_SPEND', 'DIAG_CODE_PREFIX', 'BEN_TYPE', 'DED_AMT']

# 1. 余分な特徴量を削除
# モデルが使用する特徴量のみを保持する
df = df[[col for col in expected_columns if col in df.columns]]

# 2. 不足している特徴量を追加
# 現在のデータにモデル学習時にあった特徴量が欠けている場合、デフォルト値（0）で埋める
for col in expected_columns:
    if col not in df.columns:
        df[col] = 0  # デフォルト値0で埋める

# 3. 特徴量の順序をモデル学習時と一致させる（非常に重要）
df = df[expected_columns]

# 処理後の特徴量リストを出力
print("处理后的特征：", df.columns.tolist()) #処理後の特徴量リスト

处理后的特征： ['PROV_LEVEL', 'INVOICE_CNT', 'CL_THIRD_PARTY_PAY_AMT', 'CWF_AMT_DAY', 'CODES_COUNT', 'CL_OWNER_PAY_AMT', 'PAY_AMT_USD', 'APP_AMT', 'BEN_SPEND', 'DIAG_CODE_PREFIX', 'BEN_TYPE', 'DED_AMT']


In [13]:
# 保存されたモデルをロード
model = joblib.load('best_xgboost_model.pkl')

In [14]:
# 予測を実行
predictions = model.predict(df)

# 予測結果を出力
print(predictions)

[0 0 0 0 0 0 0 0 1]


In [15]:
# 予測結果の分析
# モデルが「詐欺」と判定したサンプルを抽出し、その特徴を分析します
# 負例（詐欺と予測されたサンプル、値が1）を取得
negative_samples = df[predictions == 1]

# 負例の特徴分布統計を表示
negative_samples.describe()

# 負例（詐欺）と正例（正常）の特徴分布の比較
# 平均値(mean)と標準偏差(std)の差分を計算して、どのような傾向の違いがあるか確認
positive_samples = df[predictions == 0]
negative_samples.describe().loc[['mean', 'std']] - positive_samples.describe().loc[['mean', 'std']]

,PROV_LEVEL,INVOICE_CNT,CL_THIRD_PARTY_PAY_AMT,CWF_AMT_DAY,CODES_COUNT,CL_OWNER_PAY_AMT,PAY_AMT_USD,APP_AMT,BEN_SPEND,DIAG_CODE_PREFIX,BEN_TYPE,DED_AMT
mean,-0.875,-0.625,0.0,0.0,0.875,0.678571,-0.287884,-0.287884,-0.287884,0.25,0.0,0.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# 予測結果をDataFrameに追加
df['fraud'] = predictions

# 特徴量と目的変数（予測されたfraud）との相関関係を計算
correlation = df.corr()['fraud'].sort_values(ascending=False)

# 相関係数を出力
print(correlation)

fraud                     1.000000
CODES_COUNT               0.661438
CL_OWNER_PAY_AMT          0.553111
DIAG_CODE_PREFIX          0.053300
PAY_AMT_USD              -0.317201
APP_AMT                  -0.317201
BEN_SPEND                -0.317201
INVOICE_CNT              -0.395285
PROV_LEVEL               -0.661438
CL_THIRD_PARTY_PAY_AMT         NaN
CWF_AMT_DAY                    NaN
BEN_TYPE                       NaN
DED_AMT                        NaN
Name: fraud, dtype: float64


In [17]:
# 特徴量の重要度を取得（Feature Importance）
# モデルが予測を行う際に、どの特徴量を重視したかを確認します
importances = model.feature_importances_

# 各特徴量の重要度を出力
for i, importance in enumerate(importances):
    print(f"特征 {df.columns[i]}: 重要性 = {importance:.3f}")

特征 PROV_LEVEL: 重要性 = 0.010
特征 INVOICE_CNT: 重要性 = 0.008
特征 CL_THIRD_PARTY_PAY_AMT: 重要性 = 0.015
特征 CWF_AMT_DAY: 重要性 = 0.018
特征 CODES_COUNT: 重要性 = 0.025
特征 CL_OWNER_PAY_AMT: 重要性 = 0.017
特征 PAY_AMT_USD: 重要性 = 0.211
特征 APP_AMT: 重要性 = 0.477
特征 BEN_SPEND: 重要性 = 0.189
特征 DIAG_CODE_PREFIX: 重要性 = 0.005
特征 BEN_TYPE: 重要性 = 0.010
特征 DED_AMT: 重要性 = 0.014
